In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import numpy as np
import xgboost as xgb
from sklearn import preprocessing
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
import matplotlib.pyplot as plt
from sklearn import preprocessing, decomposition
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import confusion_matrix
import itertools
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
%matplotlib inline

In [6]:
datf=pd.read_csv('C:\Indiegogo_2020-12-18T10_43_47_067Z\Indiegogo.csv',low_memory=False)

datf.head(10)

,bullet_point,category,category_url,clickthrough_url,close_date,currency,funds_raised_amount,funds_raised_percent,image_url,is_indemand,...,perks_claimed,price_offered,price_retail,product_stage,project_id,project_type,source_url,tagline,tags,title
0,NaN,Food & Beverages,/explore/food-beverages,/projects/falls-church-distillers,2018-09-28T23:59:59-07:00,USD,200,0.005000,https://c1.iggcdn.com/indiegogo-media-prod-cld...,False,...,NaN,NaN,NaN,NaN,2402959,campaign,https://www.indiegogo.com/explore/all?project_...,We're taking our spirits to new markets across...,"[""other""]",Falls Church Distillers
1,NaN,Phones & Accessories,/explore/phones-accessories,/projects/hudway-glass-keeps-your-eyes-on-the-...,2015-11-09T23:59:59-08:00,USD,706279,6.227850,https://c1.iggcdn.com/indiegogo-media-prod-cld...,True,...,NaN,NaN,NaN,shipping,1526759,campaign,https://www.indiegogo.com/explore/all?project_...,A universal vehicle accessory turning your sma...,"[""cars""]",HUDWAY Glass: keeps your eyes on the road
2,NaN,Video Games,/explore/video-games,/projects/weathery,2017-02-26T23:59:59-08:00,USD,311,0.020733,https://c1.iggcdn.com/indiegogo-media-prod-cld...,False,...,NaN,NaN,NaN,NaN,1981307,campaign,https://www.indiegogo.com/explore/all?project_...,Survival RPG: where real weather and the time ...,"[""other""]",Weathery
3,NaN,Comics,/explore/comics,/projects/road-to-delta-dogs-season-2,2020-06-21T23:59:59-07:00,USD,1002,1.964000,https://c1.iggcdn.com/indiegogo-media-prod-cld...,True,...,NaN,NaN,NaN,NaN,2602367,campaign,https://www.indiegogo.com/explore/all?project_...,Seven cousins discover that being super-powere...,"[""indie""]",Road to Delta Dogs Season 2!
4,NaN,Culture,/explore/culture,/projects/lively-up-your-earth-eco-music-festi...,2019-01-13T23:59:59-08:00,USD,1815,0.302500,https://c1.iggcdn.com/indiegogo-media-prod-cld...,False,...,NaN,NaN,NaN,NaN,2434213,campaign,https://www.indiegogo.com/explore/all?project_...,Solitude Farm's 6th Eco-Music Festival. Bridgi...,"[""food"",""community"",""festival""]",Lively Up Your Earth Eco- Music Festival 2019
5,NaN,Home,/explore/home,/projects/neighborhood-renewal,2020-12-21T23:59:59-08:00,USD,0,0.000000,https://c1.iggcdn.com/indiegogo-media-prod-cld...,False,...,NaN,NaN,NaN,concept,2644378,campaign,https://www.indiegogo.com/explore/all?project_...,"Citizens who really care, we are people for wh...","[""festival"",""social innovations"",""safety""]",Neighborhood renewal
6,NaN,Environment,/explore/environment,/projects/art-in-the-hood-jazz-up-our-town-pro...,2017-05-12T23:59:59-07:00,USD,70,0.011667,https://c1.iggcdn.com/indiegogo-media-prod-cld...,False,...,NaN,NaN,NaN,NaN,2063976,campaign,https://www.indiegogo.com/explore/all?project_...,empowering communities towards environmental p...,"[""social innovations""]",Art in the Hood: Jazz Up Our Town Project
7,NaN,Wellness,/explore/wellness,/projects/the-giving-bowl-food-truck,2018-06-03T23:59:59-07:00,USD,1731,0.155806,https://c1.iggcdn.com/indiegogo-media-prod-cld...,False,...,NaN,NaN,NaN,NaN,2364285,campaign,https://www.indiegogo.com/explore/all?project_...,Healthy food for a happy soul! A meal is provi...,"[""kitchen"",""coffee"",""bottles"",""community"",""fem...",The Giving Bowl Food Truck
8,NaN,Health & Fitness,/explore/health-fitness,/projects/xoggl-sense-augmented-reality-medita...,2015-09-22T23:59:59-07:00,USD,460,0.009200,https://c1.iggcdn.com/indiegogo-media-prod-cld...,False,...,NaN,NaN,NaN,NaN,1390772,campaign,https://www.indiegogo.com/explore/all?project_...,Xoggl – Meditation for the Masses with a Sense...,"[""other""]",Xoggl - Sense Augmented Reality meditation device
9,NaN,Camera Gear,/explore/camera-gear,/projects/viddy-the-world-s-cutest-diy-pinhole...,2015-09-29T23:59:59-07:00,GBP,40536,2.178611,https://c1.iggcdn.com/indiegogo-media-prod-cld...,False,...,NaN,NaN,NaN,NaN,1455614,campaign,https://www.indiegogo.com/explore/all?project_...,VIDDY is the world's cutest do-it-yourself pin...,"[""other""]",VIDDY: The world's cutest DIY pi

In [18]:
print(datf['category'].apply(lambda x: len(x.split(' '))).sum())
fig = plt.figure(figsize=(12,5))
#datf.groupby('category').post.count().plot.bar(ylim=0)
#plt.show();
print('SUKASUKA')

64975
SUKASUKA


<Figure size 864x360 with 0 Axes>

In [16]:
datf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32848 entries, 0 to 32847
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   bullet_point          173 non-null    object 
 1   category              32848 non-null  object 
 2   category_url          32848 non-null  object 
 3   clickthrough_url      32848 non-null  object 
 4   close_date            32679 non-null  object 
 5   currency              32848 non-null  object 
 6   funds_raised_amount   32848 non-null  int64  
 7   funds_raised_percent  32848 non-null  float64
 8   image_url             32848 non-null  object 
 9   is_indemand           32848 non-null  bool   
 10  is_pre_launch         32848 non-null  bool   
 11  offered_by            0 non-null      float64
 12  open_date             32673 non-null  object 
 13  perk_goal_percentage  7 non-null      float64
 14  perks_claimed         7 non-null      float64
 15  price_offered      

In [19]:
def missing_data(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

missing_data(datf)

,Total,Percent
offered_by,32848,100.000000
price_retail,32848,100.000000
price_offered,32848,100.000000
perk_goal_percentage,32841,99.978690
perks_claimed,32841,99.978690
bullet_point,32675,99.473332
product_stage,29271,89.110448
open_date,175,0.532757
close_date,169,0.514491
tags,63,0.191792


In [21]:
dobuch = datf[['title','tagline','tags','category']]


,title,tagline,tags,category
0,Falls Church Distillers,We're taking our spirits to new markets across...,"[""other""]",Food & Beverages
1,HUDWAY Glass: keeps your eyes on the road,A universal vehicle accessory turning your sma...,"[""cars""]",Phones & Accessories
2,Weathery,Survival RPG: where real weather and the time ...,"[""other""]",Video Games
3,Road to Delta Dogs Season 2!,Seven cousins discover that being super-powere...,"[""indie""]",Comics
4,Lively Up Your Earth Eco- Music Festival 2019,Solitude Farm's 6th Eco-Music Festival. Bridgi...,"[""food"",""community"",""festival""]",Culture


In [46]:
dobuch = datf[  datf['title'].notnull()  ][['title','tagline','tags','category']].copy()
dobuch['fulltitle']=dobuch['title']+' '+dobuch['tagline']
dobuch.head()
ftitle=list(dobuch['fulltitle'])


In [42]:
dobuch.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32839 entries, 0 to fulltitle
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   title     32838 non-null  object
 1   tagline   32829 non-null  object
 2   tags      32775 non-null  object
 3   category  32838 non-null  object
dtypes: object(4)
memory usage: 1.3+ MB


In [48]:
dobuch.head()


,title,tagline,tags,category,fulltitle
0,Falls Church Distillers,We're taking our spirits to new markets across...,"[""other""]",Food & Beverages,Falls Church Distillers We're taking our spiri...
1,HUDWAY Glass: keeps your eyes on the road,A universal vehicle accessory turning your sma...,"[""cars""]",Phones & Accessories,HUDWAY Glass: keeps your eyes on the road A u...
2,Weathery,Survival RPG: where real weather and the time ...,"[""other""]",Video Games,Weathery Survival RPG: where real weather and ...
3,Road to Delta Dogs Season 2!,Seven cousins discover that being super-powere...,"[""indie""]",Comics,Road to Delta Dogs Season 2! Seven cousins dis...
4,Lively Up Your Earth Eco- Music Festival 2019,Solitude Farm's 6th Eco-Music Festival. Bridgi...,"[""food"",""community"",""festival""]",Culture,Lively Up Your Earth Eco- Music Festival 2019 ...


In [91]:
dobuch['fulltitle']=dobuch['fulltitle'].map(lambda x: str(x).lower())
dobuch['fulltitle']=dobuch['fulltitle'].map(lambda x: re.sub(r'[^a-zA-Z0-9]', ' ', str(x)))

In [92]:
from gensim.models import doc2vec

def label_sentences(corpus, label_type):
    """
    Gensim's Doc2Vec implementation requires each document/paragraph to have a label associated with it.
    We do this by using the TaggedDocument method. The format will be "TRAIN_i" or "TEST_i" where "i" is
    a dummy index of the complaint narrative.
    """
    labeled = []
    for i, v in enumerate(corpus):
        #print('i= ',i,' v = ',type(v.split()))
        label = label_type + '_' + str(i)
        
        labeled.append(doc2vec.TaggedDocument(str(v).split(), [label]))
                            
    return labeled

In [93]:
X_train, X_test, y_train, y_test = train_test_split(dobuch.fulltitle, dobuch.category, random_state=0, test_size=0.3)

X_train = label_sentences(X_train, 'Train')
X_test = label_sentences(X_test, 'Test')
all_data = X_train + X_test
len(all_data)
all_data[:2]
from pandas import Panel
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec

<ipython-input-93-2497fa684911>:8: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
C:\ProgramData\Miniconda3\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [98]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, min_count=1, alpha=0.065, min_alpha=0.065)
model_dbow.build_vocab([x for x in tqdm(all_data)])

100%|████████████████████████████████████████████████| 32838/32838 [00:00<00:00, 1646415.74it/s]


In [95]:
def get_vectors(model, corpus_size, vectors_size, vectors_type):
    """
    Get vectors from trained doc2vec model
    :param doc2vec_model: Trained Doc2Vec model
    :param corpus_size: Size of the data
    :param vectors_size: Size of the embedding vectors
    :param vectors_type: Training or Testing vectors
    :return: list of vectors
    """
    vectors = np.zeros((corpus_size, vectors_size))
    for i in range(0, corpus_size):
        prefix = vectors_type + '_' + str(i)
        vectors[i] = model.docvecs[prefix]
    return vectors

In [99]:
train_vectors_dbow = get_vectors(model_dbow, len(X_train), 300, 'Train')
test_vectors_dbow = get_vectors(model_dbow, len(X_test), 300, 'Test')

In [100]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(multi_class='multinomial', solver = 'lbfgs')
logreg.fit(train_vectors_dbow, y_train)

logreg.score(test_vectors_dbow, y_test)

0.038672350791717415